# 2D SWFlow - Dam break with three conical obstacles
------------------------------------------------------------------------------------------

This notebook uses Proteus to simulate a dam break over "dry land" with three obstacles using the classical Saint-Venant shallow water equations or a hyperbolic Serre--Green-Naghdi model. See [Guermond et al, 2022] for a detailed description of of the hyperbolic SGN model.

The 2D computational domain is defined as D = [0, 75m] x [0, 30m]. The topography is mostly flat except for three conical obstacles.  The water depth left of the dam (positioned at x = 16m) is 1.875m.

This example highlights the different physics of the shallow water models implemented in PROTEUS along with the capabilities to handle dry states. For more details, we refer to the user to the specific run file `dam3bumps.py` or the references listed below.

### References

- J.-L.GUERMOND, M. QUEZADA de LUNA, B. POPOV, C. KEES, M. FARTHING, Well-balanced second-order finite element approximation of the shallow water equations with friction, SIAM, J. Sci. Comput., 40:6 (2018) A3873--A3901. [https://doi.org/10.1137/17M1122463](https://doi.org/10.1137/17M1122463)

- J.-L. GUERMOND, C. KEES, B. POPOV, E. TOVAR, Hyperbolic relaxation technique for solving the dispersive Serre-       Green-Naghdi equations with topography, J. Comput. Phys., 450 (2022) 110809. [https://doi.org/10.1016/j.jcp.2021.110809](https://doi.org/10.1016/j.jcp.2021.110809)

# Running the benchmark via the terminal

The `parun` script can be to execute the python script file: `reef_island_runup.py`. There are several argument that can be supplied to the `parun` script to define various runtime options. All available options are listed when executing `parun -h` in the command line. Common command-line options are as follows:

**Option** | **Description**
:---: | :---:
 -v   | Print logging information to standard output
 -O PETSCOPTIONSFILE  | Text file of options to pass to Petsc library
 -D DATADIR | Set data directory for output storage
 -l LOGLEVEL | Store runtime information at the log level, 0 = none, 10 = everything
 -b BATCHFILENAME | Text file of auxiliary commands to execute along with main program
 -G gatherArchive | Collect data files into single file at end of simulation (will require more computational resources on large runs)
 -H hotStart | Use the last step in the archive as the initial condition and continue appending to the archive
 --SWEs | To consider SWEs applications
 
 
To run the script on more than one rank, one can invoke the following: `mpiexec -n <number of cores>` before the use of `parun` in the command line. 

## Context options for run file

Most (if not all) Proteus run files `benchmark_name.py` (in this case `reef_island_runup.py`) contain run time options specific to the model at hand. Here are some run time options for this particular example. For exact options, see the run file.

**Option** | **Description**
:---: | :---:
 sw_model | sw_model = {0,1} for {SWEs,DSWEs} 
 final_time  | Final time for simulation
 dt_output | Time interval to output solution
 still_water_depth | Depth of still water above floor
 dam_height | Height of water dam above still water 
 dam_x_location | X position of dam
 cone_amplifier | Amplification of bathymetry cone magnitudes
 
 
To modify the context options at run time, include the `-C` flag followed by `"option1=True option2=2 ..."`.

In [1]:
# Clean up previous data directory if it exists
!rm -r run_data

In [3]:
# Then we run 
!PATH=/opt/proteus/linux/bin:$PATH mpiexec -np 4 parun --SWEs dam3Bumps.py -l1 -v -C "refinement=5 final_time=20. cone_amplifier=2." -D run_data

[       0] Running Proteus version 1.8.0.dev0
Constructing SW2DCV<CompKernelTemplate<2,4,3,3,3,3>());
Constructing SW2DCV<CompKernelTemplate<2,4,3,3,3,3>());
Constructing SW2DCV<CompKernelTemplate<2,4,3,3,3,3>());
Constructing SW2DCV<CompKernelTemplate<2,4,3,3,3,3>());
2  nSpace_global
[       2] Setting initial conditions
[       3] Starting time stepping
[       3] ==============================================================
[       3] Solving over interval [ 0.00000e+00, 1.00000e-03]
[       3] ==============================================================
[       3] ==============================================================
[       3] Solving over interval [ 1.00000e-03, 1.00000e-01]
[       3] ==============================================================
[       5] ==============================================================
[       5] Solving over interval [ 1.00000e-01, 2.00000e-01]
[       5] ==============================================================
[       7] ===

## Post-process the solution using ipygany

In [4]:
# Get dependencies
import sys
sys.path.append('/opt/proteus_visualization')
from hdf5_loader import extract_arrays_metadata, extract_array
import numpy as np
from ipywidgets import Image
from ipywidgets import Play, IntSlider, HBox, link
from ipygany import Scene, Data, Component, PolyMesh, Water, UnderWater, Data, Component, Threshold
from ipydatawidgets import NDArrayWidget

In [5]:
# Load our data
arrays_metadata = extract_arrays_metadata('./run_data/dam3Bumps.h5')

mem_vertices = extract_array(arrays_metadata, 'nodesSpatial_Domain0')
vertices = np.array(mem_vertices[:, 0:2])

indices = extract_array(arrays_metadata, 'elementsSpatial_Domain0')

# This never changes, we extract it only once
bathymetry = extract_array(arrays_metadata, 'bathymetry0_t0')

# Get texture for topography
texture = Image.from_file('./wood_texture.jpg')

In [6]:
# Define simulation parameters
warp_value = 5.
num_of_steps = 200

In [ ]:
# Caching arrays on the front-end using NDArrayWidgets
h_cached = []
water_vertices_cached = []
for i in range(num_of_steps):
    h = extract_array(arrays_metadata, 'h_t{}'.format(i))

    z_water = h + bathymetry
    water_vertices = np.append(vertices, z_water.reshape((z_water.shape[0], 1)) * warp_value, axis=1).flatten()

    h_cached.append(NDArrayWidget(array=h))
    water_vertices_cached.append(NDArrayWidget(array=water_vertices))   

In [ ]:
# Set up ipygany for visualizing the solution 

h_component = Component(name='h', array=h_cached[0])

water_mesh = PolyMesh(
    vertices=water_vertices_cached[0],
    triangle_indices=indices,
    data={'h': [h_component]}
)

actual_water = Threshold(water_mesh, input='h', min=1e-3, max=1000)

floor = PolyMesh(
    vertices=np.append(vertices, bathymetry.reshape((bathymetry.shape[0], 1)) * warp_value, axis=1),
    triangle_indices=indices,
    data={'underwater': [h_component]}
)

water = Water(
    actual_water, 
    under_water_blocks=(UnderWater(floor), ),
    caustics_enabled=True
)

scene = Scene((water, ))

def update_step(change):
    i = change['new']

    h_component.array = h_cached[i]
    water_mesh.vertices = water_vertices_cached[i]

play = Play(description='Step:', min=0, max=num_of_steps-1, value=0, interval=100)
play.observe(update_step, names=['value'])

progress = IntSlider(value=0, step=1, min=0, max=num_of_steps-1)
link((progress, 'value'), (play, 'value'))

display(HBox((play, progress)))

# Visualize solution 
scene

In [ ]:
# Define some visualization parameters
water.caustics_factor = 0.20
water.under_water_blocks[0].texture = texture
scene.background_color='aliceblue'